In [1]:
!pip install rdflib

In [42]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS, SKOS
from rdflib.plugins.sparql import prepareQuery

g = Graph()
g.parse("result-with-links.ttl", format="turtle")
calle = "calle de Alcalá"

q = prepareQuery('''
            PREFIX ns: <http://smartcity.linkeddata.es/lcc/ontology/BicicletasElectricas#> 
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>   
            PREFIX data: <http://smartcity.linkeddata.es/lcc/resource/> 
            PREFIX owl: <http://www.w3.org/2002/07/owl#>  
            PREFIX bicycle: <data:BicycleStation> 
            PREFIX district: <data:District> 
            PREFIX direction: <data:Direction>  
            PREFIX hood: <data:Neighbourhood>  
            PREFIX street: <data:Street>           

            SELECT ?Key WHERE {
                ?Street rdfs:label "''' + calle + '''". 
                ?Street owl:sameAs ?Key
            } LIMIT 1''')
        
for r in g.query(q):
    prueba = r.Key
clave = prueba.split('/')

grafo = Graph()
grafo.parse("https://www.wikidata.org/wiki/Special:EntityData/Q2424746.ttl")
print(f"Número de tripletas es: {len(grafo)}")

q = grafo.query("""SELECT ?p ?label_objeto
                    WHERE 
                    {
                    wd:Q2424746 ?p ?objeto.
                    ?objeto rdfs:label ?label_objeto.
                    }LIMIT 10""")

print(f"Número de resultados: {len(q)}")


for p, label_objeto in q:
   print(f'Objeto: {label_objeto} Predicado: {p}')

Número de tripletas es: 2266
Número de resultados: 10
Objeto: street Predicado: http://www.wikidata.org/prop/direct/P31
Objeto: Category:Calle de Alcalá Predicado: http://www.wikidata.org/prop/direct/P910
Objeto: Spain Predicado: http://www.wikidata.org/prop/direct/P17
Objeto: Salamanca Predicado: http://www.wikidata.org/prop/direct/P131
Objeto: Gran Vía Predicado: http://www.wikidata.org/prop/direct/P2789
Objeto: plaza de Cibeles Predicado: http://www.wikidata.org/prop/direct/P2789
Objeto: Paseo del Prado Predicado: http://www.wikidata.org/prop/direct/P2789
Objeto: Paseo de Recoletos Predicado: http://www.wikidata.org/prop/direct/P2789
Objeto: Plaza de la Independencia Predicado: http://www.wikidata.org/prop/direct/P2789
Objeto: Calle de O'Donnell, Madrid Predicado: http://www.wikidata.org/prop/direct/P2789


In [46]:
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS, SKOS
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.parse('../data/result-with-links.ttl', format="turtle")

calle = "calle de Alcalá"
q = prepareQuery('''
            PREFIX ns: <http://smartcity.linkeddata.es/lcc/ontology/BicicletasElectricas#> 
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>   
            PREFIX data: <http://smartcity.linkeddata.es/lcc/resource/> 
            PREFIX owl: <http://www.w3.org/2002/07/owl#>   
            PREFIX bicycle: <data:BicycleStation> 
            PREFIX district: <data:District> 
            PREFIX direction: <data:Direction>  
            PREFIX hood: <data:Neighbourhood>  
            PREFIX street: <data:Street>           

            SELECT ?Key WHERE {
                ?Street rdfs:label "''' + calle + '''". 
                ?Street owl:sameAs ?Key
            } LIMIT 1'''
        )
print(f"Número de tripletas es: {len(g)}")


        ##ADAPTER##
clave = ''
for r in g.query(q):
    clave= r.Key.split('/')

        #DESCARGAMOS LA PAGINA DE WIKIDATA QUE NOS INTERESA
grafo = Graph()
grafo.parse("https://www.wikidata.org/wiki/Special:EntityData/"+clave[-1]+".ttl")

print(f"Número de tripletas es: {len(grafo)}")

        #CONSULTAMOS A ESA PAGINA DESCARGADA, CON prepareQuery ESTO NO FUNCIONA
q = grafo.query("""SELECT ?p ?label_objeto
                    WHERE 
                    {
                        wd:"""+clave[-1]+""" ?p ?objeto.
                        ?objeto rdfs:label ?label_objeto.
                    }LIMIT 5""")
        
print(f"Número de querys es: {len(q)}")
        #HACEMOS UN ADAPTER PARA RETENER LOS VALORES
valores = {}
for p, label_objeto in q:
    valores[p] = label_objeto

if len(valores) == 0:
    valores['resultado'] = "No tiene ningun link con datos de WIKIDATA"

for clave, valor in valores.items():
    print(f'Clave: {clave}  Valor: {valor}')

len(valores)

Número de tripletas es: 4655
Número de tripletas es: 2266
Número de querys es: 5
Clave: http://www.wikidata.org/prop/direct/P31  Valor: street
Clave: http://www.wikidata.org/prop/direct/P910  Valor: Category:Calle de Alcalá
Clave: http://www.wikidata.org/prop/direct/P17  Valor: Spain
Clave: http://www.wikidata.org/prop/direct/P131  Valor: Salamanca
Clave: http://www.wikidata.org/prop/direct/P2789  Valor: Gran Vía


5

In [4]:
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.parse("result-with-links.ttl", format="turtle")
distrito = "Carabanchel"

q = prepareQuery('''
            PREFIX ns: <http://smartcity.linkeddata.es/lcc/ontology/BicicletasElectricas#> 
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>   
            PREFIX data: <http://smartcity.linkeddata.es/lcc/resource/>  
            PREFIX bicycle: <data:BicycleStation> 
            PREFIX district: <data:District> 
            PREFIX direction: <data:Direction>  
            PREFIX hood: <data:Neighbourhood>  
            PREFIX street: <data:Street>           

            SELECT ?Street WHERE {
                ?District rdfs:label "''' + distrito + '''". 
                ?Hood ns:neighbourhoodBelongsDistrict ?District.                
                ?Direction ns:directionBelongsNeighbourhood ?Hood. 
                ?Direction ns:hasStreet ?Street.
            } LIMIT 25''')
        
for r in g.query(q):
    print(r.Street)

http://smartcity.linkeddata.es/lcc/resource/Street/GlorietaDelMarquesDeVadillo
http://smartcity.linkeddata.es/lcc/resource/Street/CalleDeSanDelfin
